## AVISO:
Este notebook es provisional, la versión final es training.py

In [1]:
import random
import spacy # Para Lemmatizer con lenguaje español
import json # Para 'intents.json'
import pickle
import numpy as np
import nltk # Natural Language ToolKit, que permite tokenizar

# Redes Neuronales:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
nlp = spacy.load('es_core_news_sm')
def lemmatizer(text):  
    doc = nlp(text)
    return ' '.join([word.lemma_ for word in doc])

text = 'personas'
print(lemmatizer(text))

persona


In [5]:
intents = json.loads(open('intents.json').read())

In [7]:
words = [] # Palabras individuales usadas
tags = [] # Etiquetas de intención
ignore_letters = ['¿', '?', '.', '!', '(', ')']
documents = [] # Lista de tuplas ([Lista de palabras], etiqueta asociada de intención)
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #print(pattern)
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        #print(word_list)
        #words.append(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in tags:
            tags.append(intent['tag'])
            
words = [lemmatizer(word) for word in words if word not in ignore_letters]
print(words)
print("---")
words = sorted(set(words))
words

['Hola', 'Hey', 'Hi', 'Buenos', 'dias', 'Buenas', 'tarde', 'Buenas', 'noche', 'Buenas', 'saber', 'donde', 'estã ¡', 'â¿dã³ndir', 'quedar', 'donde', 'quedar', 'este', 'lugar', 'como', 'poder', 'llegar', 'saber', 'donde', 'quedar', 'yo', 'poder', 'llevar', 'aqui', 'llevame', 'aqui']
---


['Buenas',
 'Buenos',
 'Hey',
 'Hi',
 'Hola',
 'aqui',
 'como',
 'dias',
 'donde',
 'este',
 'estã ¡',
 'llegar',
 'llevame',
 'llevar',
 'lugar',
 'noche',
 'poder',
 'quedar',
 'saber',
 'tarde',
 'yo',
 'â¿dã³ndir']